In [ ]:
!pip install transformers
!pip install datasets
!pip install seqeval

In [2]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, DataCollatorForTokenClassification, TFAutoModelForTokenClassification, create_optimizer
import tensorflow as tf
import numpy as np

In [3]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [4]:
raw_datasets = load_dataset("conll2003")
label_names = raw_datasets["train"].features["ner_tags"].feature.names
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched = True,
    remove_columns = raw_datasets["train"].column_names)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

Generating train split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3454 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Parameter 'function'=<function tokenize_and_align_labels at 0x7ff717724e60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [6]:
def full_pipeline(batch_size, init_lr, num_epochs):

    # Training data
    tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
        columns = ["attention_mask", "input_ids", "labels", "token_type_ids"],
        collate_fn = data_collator,
        shuffle = True,
        batch_size = batch_size)


    # Validation data
    tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
        columns = ["attention_mask", "input_ids", "labels", "token_type_ids"],
        collate_fn = data_collator,
        shuffle = False,
        batch_size = batch_size)


    # Create optimizer
    num_train_steps = len(tf_train_dataset) * num_epochs

    optimizer, schedule = create_optimizer(
        init_lr = init_lr,
        num_warmup_steps = 0,
        num_train_steps = num_train_steps,
        weight_decay_rate = 0.01)


    # Model
    model = TFAutoModelForTokenClassification.from_pretrained(model_checkpoint, id2label = id2label, label2id = label2id)

    model.compile(optimizer = optimizer)

    model.fit(tf_train_dataset, validation_data = tf_eval_dataset, epochs = num_epochs)


    # Check performance of the Model
    metric = load_metric("seqeval")

    all_predictions = []
    all_labels = []
    for batch in tf_eval_dataset:
        logits = model.predict(batch)["logits"]
        labels = batch["labels"]
        predictions = np.argmax(logits, axis=-1)
        for prediction, label in zip(predictions, labels):
            for predicted_idx, label_idx in zip(prediction, label):
                if label_idx == -100:
                    continue
                all_predictions.append(label_names[predicted_idx])
                all_labels.append(label_names[label_idx])
                
    return(metric.compute(predictions=[all_predictions], references=[all_labels]))

In [17]:
num_epochs = 3

# Try different parameters

### batch_size=32, init_lr=2e-5

In [7]:
batch_size = 32
init_lr = 2e-5

In [8]:
full_pipeline(batch_size, init_lr, num_epochs)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Epoch 1/3
439/439 [==============================] - 191s 388ms/step - loss: 0.2367 - val_loss: 0.0683
Epoch 2/3
439/439 [==============================] - 172s 392ms/step - loss: 0.0576 - val_loss: 0.0576
Epoch 3/3
439/439 [==============================] - 174s 397ms/step - loss: 0.0353 - val_loss: 0.0573


{'LOC': {'f1': 0.957516339869281,
  'number': 1837,
  'precision': 0.9580381471389646,
  'recall': 0.9569951007076756},
 'MISC': {'f1': 0.8562532433834976,
  'number': 922,
  'precision': 0.8208955223880597,
  'recall': 0.8947939262472885},
 'ORG': {'f1': 0.9030837004405287,
  'number': 1341,
  'precision': 0.8893709327548807,
  'recall': 0.9172259507829977},
 'PER': {'f1': 0.9636999193331541,
  'number': 1842,
  'precision': 0.9547149706979222,
  'recall': 0.9728555917480999},
 'overall_accuracy': 0.9848560664037205,
 'overall_f1': 0.9309084869622987,
 'overall_precision': 0.9188524590163935,
 'overall_recall': 0.9432850891955571}

In [14]:
result_1 = {'LOC': {'f1': 0.957516339869281,
  'number': 1837,
  'precision': 0.9580381471389646,
  'recall': 0.9569951007076756},
 'MISC': {'f1': 0.8562532433834976,
  'number': 922,
  'precision': 0.8208955223880597,
  'recall': 0.8947939262472885},
 'ORG': {'f1': 0.9030837004405287,
  'number': 1341,
  'precision': 0.8893709327548807,
  'recall': 0.9172259507829977},
 'PER': {'f1': 0.9636999193331541,
  'number': 1842,
  'precision': 0.9547149706979222,
  'recall': 0.9728555917480999},
 'overall_accuracy': 0.9848560664037205,
 'overall_f1': 0.9309084869622987,
 'overall_precision': 0.9188524590163935,
 'overall_recall': 0.9432850891955571}

In [15]:
result_1['overall_accuracy']

0.9848560664037205

### batch_size=16, init_lr=2e-5

In [16]:
batch_size = 16
init_lr = 2e-5

result_2 = full_pipeline(batch_size, init_lr, num_epochs)
result_2

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do 

Epoch 1/3
878/878 [==============================] - 201s 210ms/step - loss: 0.1678 - val_loss: 0.0645
Epoch 2/3
878/878 [==============================] - 186s 212ms/step - loss: 0.0459 - val_loss: 0.0565
Epoch 3/3
878/878 [==============================] - 187s 213ms/step - loss: 0.0264 - val_loss: 0.0572


{'LOC': {'f1': 0.9571583514099783,
  'number': 1837,
  'precision': 0.9535386277687736,
  'recall': 0.9608056614044638},
 'MISC': {'f1': 0.8824786324786325,
  'number': 922,
  'precision': 0.8694736842105263,
  'recall': 0.89587852494577},
 'ORG': {'f1': 0.9064327485380117,
  'number': 1341,
  'precision': 0.8888888888888888,
  'recall': 0.9246830723340791},
 'PER': {'f1': 0.9630026809651475,
  'number': 1842,
  'precision': 0.951271186440678,
  'recall': 0.9750271444082519},
 'overall_accuracy': 0.9853122976393713,
 'overall_f1': 0.9358057541992351,
 'overall_precision': 0.9248849441157133,
 'overall_recall': 0.9469875462807136}

### batch_size=8, init_lr=2e-5

In [18]:
batch_size = 8
init_lr = 2e-5

result_3 = full_pipeline(batch_size, init_lr, num_epochs)
result_3

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do 

Epoch 1/3
1756/1756 [==============================] - 250s 132ms/step - loss: 0.1422 - val_loss: 0.0672
Epoch 2/3
1756/1756 [==============================] - 233s 133ms/step - loss: 0.0404 - val_loss: 0.0608
Epoch 3/3
1756/1756 [==============================] - 231s 131ms/step - loss: 0.0224 - val_loss: 0.0532


{'LOC': {'f1': 0.9578833693304536,
  'number': 1837,
  'precision': 0.950187466523835,
  'recall': 0.9657049537289059},
 'MISC': {'f1': 0.8759894459102903,
  'number': 922,
  'precision': 0.8530318602261048,
  'recall': 0.9002169197396963},
 'ORG': {'f1': 0.9117647058823529,
  'number': 1341,
  'precision': 0.8992023205221175,
  'recall': 0.9246830723340791},
 'PER': {'f1': 0.9679158802911836,
  'number': 1842,
  'precision': 0.9614354579539368,
  'recall': 0.9744842562432139},
 'overall_accuracy': 0.9859598516512628,
 'overall_f1': 0.9376454938476888,
 'overall_precision': 0.926552744002629,
 'overall_recall': 0.9490070683271625}

### batch_size=32, init_lr=2e-4

In [19]:
batch_size = 32
init_lr = 2e-4

result_4 = full_pipeline(batch_size, init_lr, num_epochs)
result_4

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do 

Epoch 1/3
439/439 [==============================] - 191s 394ms/step - loss: 0.1473 - val_loss: 0.1007
Epoch 2/3
439/439 [==============================] - 175s 398ms/step - loss: 0.0482 - val_loss: 0.0659
Epoch 3/3
439/439 [==============================] - 174s 396ms/step - loss: 0.0178 - val_loss: 0.0635


{'LOC': {'f1': 0.9491985873403966,
  'number': 1837,
  'precision': 0.9473969631236443,
  'recall': 0.9510070767555797},
 'MISC': {'f1': 0.879201680672269,
  'number': 922,
  'precision': 0.8523421588594705,
  'recall': 0.9078091106290672},
 'ORG': {'f1': 0.9065974796145292,
  'number': 1341,
  'precision': 0.9012527634487841,
  'recall': 0.9120059656972409},
 'PER': {'f1': 0.9606425702811244,
  'number': 1842,
  'precision': 0.9477020602218701,
  'recall': 0.9739413680781759},
 'overall_accuracy': 0.9845764408076765,
 'overall_f1': 0.9321018472291563,
 'overall_precision': 0.9218235681369322,
 'overall_recall': 0.942611915180074}

### batch_size=32, init_lr=2e-3

In [20]:
batch_size = 32
init_lr = 2e-3

result_5 = full_pipeline(batch_size, init_lr, num_epochs)
result_5

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do 

Epoch 1/3
439/439 [==============================] - 192s 397ms/step - loss: 1.0691 - val_loss: 1.0375
Epoch 2/3
439/439 [==============================] - 173s 394ms/step - loss: 1.0269 - val_loss: 1.0266
Epoch 3/3
439/439 [==============================] - 173s 394ms/step - loss: 1.0216 - val_loss: 1.0249


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'LOC': {'f1': 0.0, 'number': 1837, 'precision': 0.0, 'recall': 0.0},
 'MISC': {'f1': 0.0, 'number': 922, 'precision': 0.0, 'recall': 0.0},
 'ORG': {'f1': 0.0, 'number': 1341, 'precision': 0.0, 'recall': 0.0},
 'PER': {'f1': 0.0, 'number': 1842, 'precision': 0.0, 'recall': 0.0},
 'overall_accuracy': 0.7612144581150292,
 'overall_f1': 0.0,
 'overall_precision': 0.0,
 'overall_recall': 0.0}

### batch_size=32, init_lr=2e-6

In [21]:
batch_size = 32
init_lr = 2e-6

result_6 = full_pipeline(batch_size, init_lr, num_epochs)
result_6

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do 

Epoch 1/3
439/439 [==============================] - 191s 395ms/step - loss: 0.8426 - val_loss: 0.4194
Epoch 2/3
439/439 [==============================] - 174s 396ms/step - loss: 0.3230 - val_loss: 0.2253
Epoch 3/3
439/439 [==============================] - 175s 399ms/step - loss: 0.2301 - val_loss: 0.1950


{'LOC': {'f1': 0.6699551569506726,
  'number': 1837,
  'precision': 0.5695768204346169,
  'recall': 0.8132825258573761},
 'MISC': {'f1': 0.036630036630036625,
  'number': 922,
  'precision': 0.11764705882352941,
  'recall': 0.021691973969631236},
 'ORG': {'f1': 0.5894346617238183,
  'number': 1341,
  'precision': 0.5031645569620253,
  'recall': 0.7114093959731543},
 'PER': {'f1': 0.9201191443271053,
  'number': 1842,
  'precision': 0.917882225823879,
  'recall': 0.9223669923995657},
 'overall_accuracy': 0.9389091658326956,
 'overall_f1': 0.6676814613042782,
 'overall_precision': 0.6371559633027523,
 'overall_recall': 0.7012790306294177}

### batch_size=32, init_lr=8e-6

In [22]:
batch_size = 32
init_lr = 8e-6

result_7 = full_pipeline(batch_size, init_lr, num_epochs)
result_7

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do 

Epoch 1/3
439/439 [==============================] - 190s 392ms/step - loss: 0.3776 - val_loss: 0.1109
Epoch 2/3
439/439 [==============================] - 173s 393ms/step - loss: 0.0992 - val_loss: 0.0800
Epoch 3/3
439/439 [==============================] - 174s 397ms/step - loss: 0.0732 - val_loss: 0.0759


{'LOC': {'f1': 0.9230355220667384,
  'number': 1837,
  'precision': 0.9127195316657797,
  'recall': 0.9335873707131193},
 'MISC': {'f1': 0.7471967380224261,
  'number': 922,
  'precision': 0.7048076923076924,
  'recall': 0.7950108459869848},
 'ORG': {'f1': 0.8602150537634409,
  'number': 1341,
  'precision': 0.8281573498964804,
  'recall': 0.8948545861297539},
 'PER': {'f1': 0.956708792686206,
  'number': 1842,
  'precision': 0.9477890250399574,
  'recall': 0.9657980456026058},
 'overall_accuracy': 0.9787926061105552,
 'overall_f1': 0.8906211536883565,
 'overall_precision': 0.8690152121697358,
 'overall_recall': 0.9133288455065635}

## batch_size=8, init_lr=1e-5

In [23]:
batch_size = 8
init_lr = 1e-5

result_8 = full_pipeline(batch_size, init_lr, num_epochs)
result_8

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do 

Epoch 1/3
1756/1756 [==============================] - 250s 132ms/step - loss: 0.1802 - val_loss: 0.0728
Epoch 2/3
1756/1756 [==============================] - 231s 132ms/step - loss: 0.0543 - val_loss: 0.0581
Epoch 3/3
1756/1756 [==============================] - 232s 132ms/step - loss: 0.0342 - val_loss: 0.0558


{'LOC': {'f1': 0.9513513513513513,
  'number': 1837,
  'precision': 0.9447128287707998,
  'recall': 0.9580838323353293},
 'MISC': {'f1': 0.8555033907146583,
  'number': 922,
  'precision': 0.8241206030150754,
  'recall': 0.8893709327548807},
 'ORG': {'f1': 0.9037472446730346,
  'number': 1341,
  'precision': 0.8906589427950761,
  'recall': 0.9172259507829977},
 'PER': {'f1': 0.9641025641025641,
  'number': 1842,
  'precision': 0.9586688137412775,
  'recall': 0.9695982627578719},
 'overall_accuracy': 0.9850032377700595,
 'overall_f1': 0.9292593822650281,
 'overall_precision': 0.917076368403802,
 'overall_recall': 0.9417704476607203}

## Results

|**Batch size**|**Initial learning rate**|**Overall accuracy**|
|:------------:|-------------------------|:------------------:|
| ========     | ===============         | =============      |
| 32           | 0.000002                | 0.9389             |
| 32           | 0.000008                | 0.9788             |
| 32           | 0.00002                 | 0.9849             |
| 32           | 0.0002                  | 0.9846             |
| 32           | 0.002                   | 0.7612             |
| ----------------- | ------------------------------ | --------------------------- |
| 16           | 0.00002                 | 0.9853             |
| **`8`**      | **`0.00002`**           | **`0.9860`**       |
| 8            | 0.00001                 | 0.9850             |

# Fit best parameters & Upload the model

In [38]:
batch_size = 8
init_lr = 2e-5
num_epochs = 5

In [39]:
# Training data
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns = ["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn = data_collator,
    shuffle = True,
    batch_size = batch_size)


# Validation data
tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns = ["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn = data_collator,
    shuffle = False,
    batch_size = batch_size)


# Create optimizer
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr = init_lr,
    num_warmup_steps = 0,
    num_train_steps = num_train_steps,
    weight_decay_rate = 0.01)


# Model
model = TFAutoModelForTokenClassification.from_pretrained(model_checkpoint, id2label = id2label, label2id = label2id)

model.compile(optimizer = optimizer)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
Some layers from the model checkpoint at nilavo-bert-finetuned were not used when initializing TFBertForTokenClassification: ['dropout_341']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificatio

In [26]:
# Login to huggingface account
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [40]:
# Fit best parameters and upload the model
from transformers.keras_callbacks import PushToHubCallback
from transformers import pipeline

callback = PushToHubCallback(output_dir="nilavo-bert-finetuned", tokenizer=tokenizer)
model.fit(tf_train_dataset, validation_data = tf_eval_dataset, callbacks = [callback], epochs = num_epochs)

/content/nilavo-bert-finetuned is already a clone of https://huggingface.co/NilavoBoral/nilavo-bert-finetuned. Make sure you pull the latest changes with `repo.git_pull()`.


Epoch 1/5


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


1756/1756 [==============================] - 275s 147ms/step - loss: 0.0171 - val_loss: 0.0676
Epoch 2/5
   1/1756 [..............................] - ETA: 5:44 - loss: 0.0119

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


1756/1756 [==============================] - 245s 140ms/step - loss: 0.0105 - val_loss: 0.0640
Epoch 3/5
1756/1756 [==============================] - 238s 135ms/step - loss: 0.0058 - val_loss: 0.0691
Epoch 4/5
1756/1756 [==============================] - 233s 133ms/step - loss: 0.0031 - val_loss: 0.0733
Epoch 5/5
1756/1756 [==============================] - 233s 133ms/step - loss: 0.0018 - val_loss: 0.0755


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 3.34k/411M [00:00<?, ?B/s]

To https://huggingface.co/NilavoBoral/nilavo-bert-finetuned
   fbd3365..1a1231b  main -> main



# Predict for a new test data

In [49]:
# Predict for a new input
model_checkpoint = "nilavo-bert-finetuned"
token_classifier = pipeline("token-classification", model = model_checkpoint, aggregation_strategy = "simple")
output = token_classifier(["My name is Nilavo Boral and I work at CloudCraftz in Kolkata.", 
                           "I am pursuing Masters in Data Science from Kalyani University, Kalyani, West Bengal."])
output

Some layers from the model checkpoint at nilavo-bert-finetuned were not used when initializing TFBertForTokenClassification: ['dropout_493']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at nilavo-bert-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


[[{'end': 23,
   'entity_group': 'PER',
   'score': 0.9997692,
   'start': 11,
   'word': 'Nilavo Boral'},
  {'end': 49,
   'entity_group': 'ORG',
   'score': 0.9996932,
   'start': 38,
   'word': 'CloudCraftz'},
  {'end': 60,
   'entity_group': 'LOC',
   'score': 0.99945027,
   'start': 53,
   'word': 'Kolkata'}],
 [{'end': 61,
   'entity_group': 'ORG',
   'score': 0.996344,
   'start': 43,
   'word': 'Kalyani University'},
  {'end': 70,
   'entity_group': 'LOC',
   'score': 0.999254,
   'start': 63,
   'word': 'Kalyani'},
  {'end': 83,
   'entity_group': 'LOC',
   'score': 0.99978316,
   'start': 72,
   'word': 'West Bengal'}]]

In [54]:
final_output = []

for sentence in output:
    semi_final_output = []
    for i in np.arange(0, len(sentence)):
        semi_final_output.append({sentence[i]['word'] : sentence[i]['entity_group']})
    final_output.append(semi_final_output)

final_output

[[{'Nilavo Boral': 'PER'}, {'CloudCraftz': 'ORG'}, {'Kolkata': 'LOC'}],
 [{'Kalyani University': 'ORG'}, {'Kalyani': 'LOC'}, {'West Bengal': 'LOC'}]]